# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')

Визуально осмотрим данные

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


В датасете нет пропусков, проверим на дубликаты 

In [4]:
df.duplicated().sum()

153

Удалим дубликаты

In [5]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

Дубликаты удалены, можно переходить к дальнейшей работе

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Предсказания $a$ не изменится

**Обоснование:** Представим выражение как $w_{new} = ((XP)^T XP)^{-1} (XP)^T y$, где $XP$ - это произведение признаков $X$ на обратимую матрицу $P$, а $a_{new} = XPw_{new}$
$$ w_{new} = ((XP)^T XP)^{-1} (XP)^T y = \\\\
(P^T X^T X P)^{-1} P^T X^T y =\\\\ 
((P^T X^T X) P)^{-1} P^T X^T y = \\\\
P^{-1} (P^T X^T X)^{-1} P^T X^T y = \\\\
P^{-1} (P^T (X^T X))^{-1} P^T X^T y =\\\\ 
P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = \\\\
P^{-1} (X^T X)^{-1} ((P^T)^{-1} P^T) X^T y =\\\\ 
P^{-1} (X^T X)^{-1} E X^T y = \\\\
P^{-1} ((X^T X)^{-1} X^T y) = \\\\
P^{-1} w
$$

Подставим выражение в формулу $ a_{new} $

$$ a_{new} = XP P^{-1}w = Xw $$

$$ a_{new} = a $$

## Алгоритм преобразования

**Алгоритм**

Возьмем случайно сгенерированную матрицу $M$ размера $n*n$, определим  по числу признаков в $X$

Проверим существование обратной матрицы $M$

Умножим исходную матрицу $X$ на матрицу $M$.

Для проверки определим метрику R2, и сравним результат на оригинальном и преобразованном датасетах.

**Обоснование**

Согласно доказательству выше, при умножении матрицы признаков $X$ на произвольную, обратимую матрицу $M$, качество линейной регрессии останется неизменным.

In [7]:
class MatrixTransformation():
    def fit(self, x, seed=42):
        
        np.random.seed(seed)
        n = x.shape[1]

        try:
            M = np.random.normal(size=(n, n))
            np.linalg.inv(M)
        except: 
            M = np.random.normal(size=(n, n))
            np.linalg.inv(M)

        self.M = M
        self.inv = np.linalg.inv(M)
        return self.M
        
    def transform(self, x):
        return x @ self.M
    
    def inverse_transform(self, X):
        return X @ self.inv

## Проверка алгоритма

Разделим данные на признаки и целевую переменную

In [9]:
x = df.drop(['Страховые выплаты'], axis=1)
y = df['Страховые выплаты']

Преобразуем признаки

In [10]:
Mt = MatrixTransformation()
M = Mt.fit(x)
x_new = Mt.transform(x) 

Сравним размеры признаков до и после преобразование, а так же взглянем на получившиеся данные

In [11]:
display(x_new.shape, x.shape, x_new.head())

(4847, 4)

(4847, 4)

,0,1,2,3
0,-23294.791154,26899.327002,-22921.847067,-23067.770211
1,-17850.555758,20604.598076,-17538.953455,-17662.990926
2,-9865.752553,11386.970944,-9685.974377,-9758.069218
3,-19581.515190,22616.010381,-19294.804157,-19405.939170
4,-12259.341053,14154.123038,-12050.336135,-12132.535366


Данные закодированы, перейдем к обучению моделей и сравнению данных 

In [12]:
lr = LinearRegression()

lr.fit(x, y)
predict1 = lr.predict(x)

lr.fit(x_new, y)
predict2 = lr.predict(x_new)

print(r2_score(y, predict1), r2_score(y, predict2))
print(f'Метрики равны: {np.isclose(r2_score(y, predict1), r2_score(y, predict2))}')

0.4302010044852067 0.43020100448521004
Метрики равны: True


Мы видим что метрики одинаковые

Попробуем преобразовать наши данные в первоначальный вид

In [13]:
x_final = Mt.inverse_transform(x_new)
x_final.head()

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,1.168548e-11,46.0,38000.0,1.000000e+00
2,4.429902e-12,29.0,21000.0,1.109661e-12
3,9.105370e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,2.769852e-13


Сравним получившиеся данные с первоначальными 

In [14]:
x.describe(), x_final.describe()

(               Пол      Возраст      Зарплата  Члены семьи
 count  4847.000000  4847.000000   4847.000000  4847.000000
 mean      0.498453    31.023932  39895.811842     1.203425
 std       0.500049     8.487995   9972.953985     1.098664
 min       0.000000    18.000000   5300.000000     0.000000
 25%       0.000000    24.000000  33200.000000     0.000000
 50%       0.000000    30.000000  40200.000000     1.000000
 75%       1.000000    37.000000  46600.000000     2.000000
 max       1.000000    65.000000  79000.000000     6.000000,
                   0            1             2             3
 count  4.847000e+03  4847.000000   4847.000000  4.847000e+03
 mean   4.984527e-01    31.023932  39895.811842  1.203425e+00
 std    5.000492e-01     8.487995   9972.953985  1.098664e+00
 min    1.421267e-12    18.000000   5300.000000 -1.769781e-12
 25%    8.826456e-12    24.000000  33200.000000  1.875395e-12
 50%    1.832487e-11    30.000000  40200.000000  1.000000e+00
 75%    1.000000e+00    3

Данные идентичны за исключением ошибки в матричных вычислениях, это значит что алгоритм успешно работает 

### Вывод

В ходе работы мы изучили и предобработали данные, далее мы провели реализацию алгоритма кодирования признаков и убедились, что данный алгоритм не влияет на качество модели - данный инструмент можно использовать в дальнейшем для кодирования признаков.